<a href="https://colab.research.google.com/github/cedro3/mediapipe/blob/main/mediapipe_for_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# セットアップ

In [ ]:
# ライブラリーのインストール
!pip install numpy==1.19.3
!pip install mediapipe

# github からコードをコピー
!git clone https://github.com/cedro3/mediapipe.git
%cd mediapipe/

In [ ]:
# 初期設定
import mediapipe as mp
mp_holistic = mp.solutions.holistic

# Initialize MediaPipe Holistic.
holistic = mp_holistic.Holistic(
    static_image_mode=True, min_detection_confidence=0.5)

# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# サンプルビデオの読み込み＋静止画に分解

In [ ]:
# ビデオの指定
video_path = './video/mana.mp4'

In [ ]:
# video2images
import os
import shutil
import cv2

# 既にimagesフォルダーがあれば削除
if os.path.isdir('images'):
    shutil.rmtree('images')

os.makedirs('images', exist_ok=True)
 
def video_2_images(video_file= video_path,   
                   image_dir='./images/', 
                   image_file='img_%s.png'):
 
    # Initial setting
    i = 0
    interval = 3
    length = 300
    
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()  
        if flag == False:  
                break
        if i == length*interval:
                break
        if i % interval == 0:    
           cv2.imwrite(image_dir+image_file % str(i).zfill(6), frame)
        i += 1 
    cap.release()  
 
def main():
    video_2_images()
    
if __name__ == '__main__':
    main()

# MediaPipeで静止画を処理

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import glob

# image file names to files in list format
files=[]
for name in sorted(glob.glob('./images/*.png')):
    files.append(name)

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in files}

for name, image in images.items():
  # Convert the BGR image to RGB and process it with MediaPipe Pose.
  results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

  # Draw pose landmarks.
  annotated_image = image.copy()
  mp_drawing.draw_landmarks(annotated_image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(annotated_image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.face_landmarks, 
      connections=mp_holistic.FACE_CONNECTIONS,
      landmark_drawing_spec=drawing_spec,
      connection_drawing_spec=drawing_spec)
  mp_drawing.draw_landmarks(
      image=annotated_image, 
      landmark_list=results.pose_landmarks, 
      connections=mp_holistic.POSE_CONNECTIONS,
      landmark_drawing_spec=drawing_spec,
      connection_drawing_spec=drawing_spec)
  cv2.imwrite(name, annotated_image)  

# 処理した画像をgif動画に変換

In [ ]:
# images2gif
from PIL import Image
import glob
 
files = sorted(glob.glob('./images/*.png'))
images = list(map(lambda file: Image.open(file), files))
images[0].save('./out.gif', save_all=True, 
               append_images=images[1:], 
               duration=100, loop=0)

In [ ]:
# display gif
from IPython.display import Image
Image('./out.gif', format='png')